# Chapter 7 文書のクラス分類

In [1]:
import common_func as cf

## 7.1 機械学習の枠組み

## 7.2 ナイーブベイズ

In [4]:
# 書籍紹介の文書ベクトルの作成
from gensim import matutils

# 書籍紹介文のデータから訓練データ用の文書ベクトルを作成する。
# TF・IDFの計算は青空文庫のデータを使用
book_texts = [cf.get_string_from_file('irpb-files/data/ch07/%d.txt' % i) for i in range(15)]
tfidf_model,dic,vectors = cf.get_tfidfmodel_and_weights(book_texts)

# denseな文書ベクトルに変換
vectors = matutils.corpus2dense(vectors,len(dic)).T

# 書籍のタイトル、クラス番号をそれぞれbook-titles.txtとclass.txtから読み込む
titles = cf.get_list_from_file('irpb-files/data/ch07/book-titles.txt')
classes = cf.get_list_from_file('irpb-files/data/ch07/class.txt')

    

In [5]:
# ナイーブベイズによる学習
from sklearn.naive_bayes import MultinomialNB

# 多項分布モデルの分類機を生成する
mnb = MultinomialNB()

# 分類機にデータと正解を与え学習させる
mnb.fit(vectors,classes)

# 分類対象のフレーズ
test_texts = ['Pythonで実装','微分方程式を解く','規格に準拠','アナログからデジタルへ','人工知能']

# フレーズの文書ベクトルを作成する
test_bows = cf.get_bows(test_texts,dic)
test_vectors = cf.get_weights(test_bows,dic,tfidf_model)
test_vectors = matutils.corpus2dense(test_vectors,len(dic)).T

# クラスを推定
predicted_classes = mnb.predict(test_vectors)

# フレーズと推定結果を並べて表示
for i,j in zip(test_texts,predicted_classes):
    print('{}: {}'.format(i,j))

Pythonで実装: 3
微分方程式を解く: 2
規格に準拠: 1
アナログからデジタルへ: 1
人工知能: 3


In [18]:
#評価のためのデータを読み込む
cv_texts = [cf.get_string_from_file('irpb-files/data/ch07/cv/%d.txt' % i) for i in range(90)]
tfidf_models,dic,vectors = cf.get_tfidfmodel_and_weights(cv_texts)
vectors = matutils.corpus2dense(vectors,len(dic)).T

#書籍のクラスを読み込む
classes = cf.get_list_from_file('irpb-files/data/ch07/cv/class.txt')

In [19]:
# ナイーブベイズの性能評価
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB

# 分割数が３の層化Ｋ分割交差検証のオブジェクトを生成する。
K = 3
skf = StratifiedKFold(n_splits=K)

# 分類器にはナイーブベイズを使用する。
classifier = MultinomialNB()

# 交差検証を実行
scores = cross_val_score(classifier,vectors,classes,cv=skf)

# 各回のスコアと平均を表示する
for i in range(K):
    print('Test %d/%d:\t%.4f' % (i+1,K,scores [i]))
print('Average:\t%.4f' % (sum(scores)/K))

Test 1/3:	0.4667
Test 2/3:	0.5333
Test 3/3:	0.5667
Average:	0.5222


In [20]:
# サポートベクトルマシンの性能評価
from sklearn import svm

K = 3
skf = StratifiedKFold(n_splits=K)

# 分類器にサポートベクトルマシンを使用する
classifier = svm.SVC(kernel='rbf',C=1,gamma=1)

# 交差検証を実行
scores = cross_val_score(classifier,vectors,classes,cv=skf)

# 結果の表示
for i in range(K):
    print('Test %d/%d:\t%.4f' % (i+1,K,scores[i]))
print('Average:\t%.4f' % (sum(scores) / K))

Test 1/3:	0.6333
Test 2/3:	0.6333
Test 3/3:	0.6333
Average:	0.6333


In [21]:
# 混同行列の計算
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

classifier = svm.SVC(kernel='rbf',C=1,gamma=1)

# 交差検証を行い、分類結果を取得する。
prediction = cross_val_predict(classifier,vectors,classes,cv=skf)

# 分類結果から混同行列を生成
cm = confusion_matrix(classes,prediction)

# classesの重複を取り除いてクラス名のリストを作成する
class_names = [j for i,j in enumerate(classes) if not j in classes[:i]]

# '%2d\t' + タブが６個(クラスの数)並んだ書式の作成
fmt = '%2d\t' * 6

# 混同行列の各行とクラス名を並べて表示する
for i,j in enumerate(cm):
    print(fmt % tuple(j),class_names[i])
    

 9	 2	 0	 3	 1	 0	 経営工学
 5	 3	 0	 3	 4	 0	 土木
 0	 1	13	 1	 0	 0	 建築
 2	 0	 0	12	 1	 0	 機械
 1	 2	 1	 1	10	 0	 電気
 0	 2	 0	 3	 0	10	 情報


In [22]:
# グリッドサーチによるパラメータの選出

from sklearn.model_selection import GridSearchCV

# パラメータの選択肢
params = {
    'kernel': ['rbf'],
    'C':[0.1,1,10,100],
    'gamma':[0.1,1,10,100]
}

classifier = svm.SVC()

# グリッドサーチを行う。分割数３の交差検証で評価する。
gs = GridSearchCV(classifier,params,cv=3)
gs.fit(vectors,classes)

# 最もよい性能を与えるものを出力する
print(gs.best_params_)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
